In [40]:
import numpy as np
import tensorflow as tf
import time
import opendatasets as od
import pandas
from PIL import Image
import glob
import matplotlib.pyplot as plt
import functools
import random
from tqdm import tqdm

In [5]:
od.download("https://www.kaggle.com/datasets/omkargurav/face-mask-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: beta10
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/omkargurav/face-mask-dataset


100%|██████████| 163M/163M [00:04<00:00, 37.7MB/s]


In [61]:
with_mask_path = '/content/face-mask-dataset/data/with_mask/*'
without_mask_path = '/content/face-mask-dataset/data/without_mask/*'

# need to randomize the images within themselves
# mask -> 1, without mask -> 0
# train_images = np.array([np.array(Image.open(fname).resize((64, 64))) for fname in glob.glob(with_mask_path)])

images = []
labels = []

# with mask
for file in glob.glob(with_mask_path):
  np_image = np.array(Image.open(file).resize((64, 64)))

  if np_image.shape == (64, 64, 3):
    images.append(np_image)
    labels.append(np.array([1]))

# without mask
for file in glob.glob(without_mask_path):
  np_image = np.array(Image.open(file).resize((64, 64)))

  if np_image.shape == (64, 64, 3):
    images.append(np_image)
    labels.append(np.array([0]))

images = np.array(images)
labels = np.array(labels)


In [68]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

images, labels = unison_shuffled_copies(images, labels)
images = images.astype('float32')
labels = labels.astype('float32')
print(images.shape)
print(labels.shape)
print(labels)

(7544, 64, 64, 3)
(7544, 1)
[[1.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [1.]]


In [73]:
def get_batch(size):
  batch_images = []
  batch_labels = []
  for i in range(size):
    index = random.randint(0, 7543)
    batch_images.append(images[index])
    batch_labels.append(labels[index])
  return (np.array(batch_images), np.array(batch_labels))


In [74]:
### Define the CNN model ###

n_filters = 12 # base number of convolutional filters

def make_standard_classifier(n_outputs=1):
  Conv2D = functools.partial(tf.keras.layers.Conv2D, padding='same', activation='relu')
  BatchNormalization = tf.keras.layers.BatchNormalization
  Flatten = tf.keras.layers.Flatten
  Dense = functools.partial(tf.keras.layers.Dense, activation='relu')

  model = tf.keras.Sequential([
    Conv2D(filters=1*n_filters, kernel_size=5,  strides=2),
    BatchNormalization(),

    Conv2D(filters=2*n_filters, kernel_size=5,  strides=2),
    BatchNormalization(),

    Conv2D(filters=4*n_filters, kernel_size=3,  strides=2),
    BatchNormalization(),

    Conv2D(filters=6*n_filters, kernel_size=3,  strides=2),
    BatchNormalization(),

    Flatten(),
    Dense(512),
    Dense(n_outputs, activation=None),
  ])
  return model

standard_classifier = make_standard_classifier()

In [75]:
### Train the standard CNN ###

# Training hyperparameters
params = dict(
  batch_size = 32,
  num_epochs = 2,  # keep small to run faster
  learning_rate = 5e-4,
  train_size = 5000,
)


optimizer = tf.keras.optimizers.Adam(params["learning_rate"]) # define our optimizer
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

@tf.function
def standard_train_step(x, y):
  with tf.GradientTape() as tape:
    # feed the images into the model
    logits = standard_classifier(x)
    # Compute the loss
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)

  # Backpropagation
  grads = tape.gradient(loss, standard_classifier.trainable_variables)
  optimizer.apply_gradients(zip(grads, standard_classifier.trainable_variables))
  return loss

# The training loop!
step = 0
for epoch in range(params["num_epochs"]):
  for idx in tqdm(range(params["train_size"] // params["batch_size"])):
    # Grab a batch of training data and propagate through the network
    x, y = get_batch(params["batch_size"])
    loss = standard_train_step(x, y)

    step += 1

100%|██████████| 156/156 [00:10<00:00, 15.59it/s]


In [78]:
### Evaluation of standard CNN ###

# TRAINING DATA
# Evaluate on a subset of CelebA+Imagenet
(batch_x, batch_y) = get_batch(2500)
y_pred_standard = tf.round(tf.nn.sigmoid(standard_classifier.predict(batch_x)))
acc_standard = tf.reduce_mean(tf.cast(tf.equal(batch_y, y_pred_standard), tf.float32))

print("Standard CNN accuracy on training set: {:.4f}".format(acc_standard.numpy()))

79/79 [==============================] - 2s 24ms/step
Standard CNN accuracy on training set: 0.9276
